In [1]:
from __future__ import annotations

from collections.abc import Callable, Sequence
from time import sleep
from typing import Any, Literal
from functools import partial
import itertools

import networkx as nx
import numpy as np
from numpy.typing import NDArray
from qokit.maxcut import maxcut_obj, get_adjacency_matrix
from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.qaoa_objective_portfolio import get_qaoa_portfolio_objective
from qokit.utils import precompute_energies
from qokit.portfolio_optimization import portfolio_brute_force
import pandas as pd
import yfinance as yf


def create_portfolio_instance(
    start_date: str = "2015-01-01",
    end_date: str = "2019-12-31",
    num_assets: int = 0,
    return_dtype: str = "numpy",
    log_returns: bool = False,
    tickers: list | None = None,
    sort_by_avg_volume: bool = False,
    seed: int = 42,
) -> (
    tuple[np.ndarray, np.ndarray]
    | tuple[pd.core.frame.DataFrame, pd.core.frame.DataFrame]
):
    """
    Parameters
    ------------------
        start_date (str): The starting date
        end_date (str): The ending date
        num_assets (int): The number of assets


        Optional:
        return_dtype (str): The return datatype for the correlation matrix
                     This is either numpy or panda

        tickers (list): The list of ticker symbols. If this is not provided, you will get
        the targets and correlation for the random tickers and you can read the
        symbols via accessing the correlation as a pandas dataframe

        log_returns: If we want the log returns instead of just returns

    -------------------
    Returns
    ---------
    List(returns, correlation) : type List(np.array, np.array or pd.dataframe)

    """

    if (
        return_dtype == "numpy"
        or return_dtype == "np"
        or return_dtype == "pd"
        or return_dtype == "pandas"
    ):
        pass
    else:
        # print(return_dtype)
        raise ValueError(
            "The return dtype should be either (numpy) or (np) or pandas or (pd)"
        )

    if tickers is None:
        # If the user doesn't provide the ticker symbols, we sort by average trading volume and just
        # give the data for the number of assets provided

        # Download data for the time period
        sp500_tickers = pd.read_html(
            "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        )[0]

        ticker_symbols = sp500_tickers["Symbol"].tolist()

        if num_assets > 0 and not sort_by_avg_volume:
            rng = np.random.default_rng(seed)
            random_tickers = rng.choice(
                len(ticker_symbols), size=num_assets, replace=False
            )

            top_N_ticker_symbols = np.array(ticker_symbols)[random_tickers].tolist()

        else:
            top_N_ticker_symbols = ticker_symbols

        # Retrieve historical price data for each ticker symbol
        stock_data = {}

        invalid_ticker_symbols = []

        for symbol in top_N_ticker_symbols:
            ticker = yf.Ticker(symbol)
            data = ticker.history(start=start_date, end=end_date)

            if data.empty:
                ### Removing the ticker symbols that are invalid
                invalid_ticker_symbols.append(symbol)
            else:
                stock_data[symbol] = data

        if sort_by_avg_volume or num_assets == 0:
            # Calculate average trading volume for each stock
            average_volumes = {}
            for symbol, data in stock_data.items():
                average_volume = data["Volume"].mean()
                average_volumes[symbol] = average_volume

            # Sort the stocks based on average trading volume
            sorted_stocks = sorted(
                average_volumes.items(), key=lambda x: x[1], reverse=True
            )

            # Extract the ticker symbols from the sorted list
            ticker_list = [tick[0] for tick in sorted_stocks]

            if num_assets > 0:
                top_N_ticker_symbols = ticker_list[:num_assets]

    ### This is if we are given a set of tickers
    ### Then just download data for these, and compute correlation matrices and return vector

    else:
        num_assets = len(tickers)
        # Retrieve historical price data for each ticker symbol
        stock_data = {}

        invalid_ticker_symbols = []

        for symbol in tickers:
            ticker = yf.Ticker(symbol)
            while True:
                data = ticker.history(start=start_date, end=end_date)
                if data.empty:
                    sleep(2)
                    continue
                else:
                    stock_data[symbol] = data

        top_N_ticker_symbols = tickers

    top_N_ticker_symbols = list(stock_data.keys())

    ### List of dataframes with the closing prices
    list_df_close = [stock_data[symbol]["Close"] for symbol in top_N_ticker_symbols]

    df_close = pd.concat(list_df_close, axis=1, keys=top_N_ticker_symbols)

    ### Drop the ones with rows with NaN values
    df_close = df_close.dropna(axis=1)
    df_close = df_close.dropna(axis=0)

    if not log_returns:
        df_returns = df_close.pct_change()  # .dropna( )

        #### Drop the first row since we have NaN's
        df_returns = df_returns.iloc[1:]

        df_mean_returns = df_returns.mean(axis=0)

        correlation_matrix = df_returns.corr()

        if return_dtype in ("pandas", "pd"):
            return [df_mean_returns, correlation_matrix]
        else:
            return [df_mean_returns.to_numpy(), correlation_matrix.to_numpy()]

    elif log_returns:
        df_logreturns = np.log(df_close / df_close.shift(1)).dropna()
        df_meanlog_returns = df_logreturns.mean(axis=0)

        correlation_matrix = df_logreturns.corr()

        if return_dtype in ("pandas", "pd"):
            return [df_meanlog_returns, correlation_matrix]

        else:
            return [df_meanlog_returns.to_numpy(), correlation_matrix.to_numpy()]


def get_data(N, seed=1, real=False):
    """
    load portofolio data from qiskit-finance (Yahoo)
    https://github.com/Qiskit/qiskit-finance/blob/main/docs/tutorials/11_time_series.ipynb
    """
    import datetime

    from qiskit_finance.data_providers import RandomDataProvider, YahooDataProvider

    tickers = []
    for i in range(N):
        tickers.append("t" + str(i))
    if real is False:
        data = RandomDataProvider(
            tickers=tickers,
            start=datetime.datetime(2016, 1, 1),
            end=datetime.datetime(2016, 1, 30),
            seed=seed,
        )
    else:
        stock_symbols = [
            "AAPL",
            "GOOGL",
            "AMZN",
            "MSFT",
            "TSLA",
            "NFLX",
            "NVDA",
            "JPM",
            "V",
            "JNJ",
            "WMT",
            "PG",
            "MA",
            "UNH",
            "HD",
            "DIS",
            "BRK-B",
            "VZ",
            "KO",
            "MRK",
            "INTC",
            "CMCSA",
            "PEP",
            "PFE",
            "CSCO",
            "XOM",
            "BA",
            "MCD",
            "ABBV",
            "IBM",
            "GE",
            "MMM",
        ]

        # switch to Atithi's implementation
        # rng = np.random.default_rng(seed)
        # date = rng.integers(0, 60)

        date = seed
        year = 2015 + date // 12
        month = date % 12 + 1
        start_date = f"{year}-{month}-01"
        end_date = f"{year}-{month}-28"
        return create_portfolio_instance(
            start_date,
            end_date,
            0,
            log_returns=True,
            seed=seed,
            # tickers=[
            #     stock_symbols[i] for i in rng.choice(len(stock_symbols), size=N, replace=False)
            # ],
            tickers=stock_symbols[:N],
        )

        # data = YahooDataProvider(
        #     tickers=stock_symbols[:N],
        #     start=datetime.datetime(2020, 1, 1),
        #     end=datetime.datetime(2020, 1, 30),
        #     # end=datetime.datetime(2021, 1, 1),
        # )

    data.run()
    # use get_period_return_mean_vector & get_period_return_covariance_matrix to get return!
    # https://github.com/Qiskit/qiskit-finance/blob/main/docs/tutorials/01_portfolio_optimization.ipynb
    means = data.get_period_return_mean_vector()
    cov = data.get_period_return_covariance_matrix()
    return means, cov


def get_real_problem(N, K, q, seed=1, pre=False):
    po_problem = {}
    po_problem["N"] = N
    po_problem["K"] = K
    po_problem["q"] = q
    po_problem["real"] = True
    po_problem["means"], po_problem["cov"] = get_data(N, seed, real=True)
    po_problem["pre"] = pre
    scale = 1
    if pre == "constant":
        scale = abs(1 / sum(po_problem["means"]))
    elif np.isscalar(pre):
        scale = pre

    po_problem["scale"] = scale
    po_problem["means"] = scale * po_problem["means"]
    po_problem["cov"] = scale * po_problem["cov"]

    return po_problem


def load_problem(
    problem: Literal["maxcut", "po"], n: int, seed: int
) -> tuple[dict[str, Any] | nx.Graph, NDArray[np.float_]]:
    if problem == "maxcut":
        return load_maxcut_problem(n, seed)
    if problem == "po":
        return load_po_problem(n, seed)
    raise ValueError(f"Problem {problem} not recognized")


def sample_gaussian_mixture(
    num_samples: int, components: Sequence[dict[str, float]], seed: int
) -> NDArray[np.float_]:
    rng = np.random.default_rng(seed)
    samples = []
    for _ in range(num_samples):
        component = rng.choice(components, p=[c["weight"] for c in components])
        sample = rng.normal(component["mean"], component["std_dev"])
        samples.append(sample)
    return np.array(samples)


def load_maxcut_problem(n: int, seed: int) -> tuple[nx.Graph, NDArray[np.float_]]:
    g = nx.random_regular_graph(3, n, seed)

    # Define the parameters for the Gaussian components
    component1 = {"mean": 0, "std_dev": 1, "weight": 0.5}
    component2 = {"mean": 5, "std_dev": 2, "weight": 0.3}
    component3 = {"mean": 10, "std_dev": 1, "weight": 0.2}
    components = [component1, component2, component3]
    weights = sample_gaussian_mixture(3 * n // 2, components, seed)
    # generate random weights
    # weights = np.random.uniform(0, 10, g.number_of_edges())
    # rescale following the rule in Eq. 6 of https://arxiv.org/pdf/2305.15201.pdf
    weights = weights / np.sqrt(np.mean(weights**2))

    for i, (w, v) in enumerate(g.edges):
        g.edges[w, v]["weight"] = weights[i]

    return g, precompute_energies(partial(maxcut_obj, w=get_adjacency_matrix(g)), n)


def load_po_problem(n, seed):
    k = n // 2
    po_problem = get_real_problem(n, k, 0.5, seed, pre=1)
    means_in_spins = np.array(
        [
            po_problem["means"][i] - po_problem["q"] * np.sum(po_problem["cov"][i, :])
            for i in range(len(po_problem["means"]))
        ]
    )
    scale = 1 / (
        np.sqrt(np.mean(((po_problem["q"] * po_problem["cov"]) ** 2).flatten()))
        + np.sqrt(np.mean((means_in_spins**2).flatten()))
    )
    po_problem["scale"] = scale
    po_problem["means"] = scale * po_problem["means"]
    po_problem["cov"] = scale * po_problem["cov"]
    (
        min_constrained,
        min_x,
        max_constrained,
        max_x,
        mean_constrained,
    ) = portfolio_brute_force(po_problem, True)
    po_problem["feasible_min"] = min_constrained
    po_problem["feasible_max"] = max_constrained
    po_problem["feasible_min_x"] = min_x
    po_problem["feasible_max_x"] = max_x
    po_problem["feasible_mean"] = mean_constrained

    return po_problem, None


def get_evaluate_energy(
    problem: dict[str, Any] | nx.Graph,
    precomputed_energies: NDArray[np.float_],
    p: int,
    objective: str = "expectation",
    simulator: str = "auto",
) -> Callable:
    if isinstance(problem, nx.Graph):
        beta_scaling = 1 / 4
        func = get_qaoa_maxcut_objective(
            problem.number_of_nodes(),
            p,
            problem,
            precomputed_energies,
            objective=objective,
            simulator=simulator,
        )
    else:
        beta_scaling = 1 / 8
        func = get_qaoa_portfolio_objective(
            problem,
            p,
            precomputed_energies=precomputed_energies,
            objective=objective,
            simulator=simulator,
        )

    def f(params):
        params = np.array(params)
        params[len(params) // 2 :] *= beta_scaling
        return func(params)

    return f


def eval_point(point, eval_func, optimal_metric, sense):
    mean = eval_func(point)
    ar = (
        sense
        * (mean - optimal_metric[int((sense + 1) / 2)])
        / (optimal_metric[0] - optimal_metric[1])
    )
    return ar

In [3]:
import nlopt
from qokit.parameter_utils import get_fixed_gamma_beta, get_sk_gamma_beta


simulator = "python"
problem = "po"
p = 5
qubit_pool = [12]
method = "LN_COBYLA"
seed_pool = list(range(60))
budget = 10000
maxfev = 2 * p + 1 + 3 
shots = budget // maxfev
rhobeg= 0.1

for i, n in enumerate(qubit_pool):
    instances = []
    results = {}
    initial_ar = []
    for j, seed in enumerate(seed_pool):
        instance, precomputed_energies = load_problem(problem, n, seed)
        instances.append((instance, precomputed_energies))
        if problem == "po":
            sense = 1
            beta_scaling = 8
            # initial_point = [-1.24727193, 1.04931211 * 8]
            gamma, beta = get_sk_gamma_beta(p)
            minval, maxval = instance["feasible_min"], instance["feasible_max"]
        elif problem == "skmodel":
            sense = -1
            beta_scaling = 4
            gamma, beta = get_sk_gamma_beta(p)
            minval, maxval = np.min(precomputed_energies), np.max(
                precomputed_energies
            )
        else:
            sense = -1
            beta_scaling = 4
            gamma, beta, ar = get_fixed_gamma_beta(3, p, True)
            gamma, beta = np.array(gamma), np.array(beta)
            minval, maxval = np.min(precomputed_energies), np.max(
                precomputed_energies
            )
        beta *= beta_scaling
        initial_point = np.concatenate((gamma, beta))

        eval_func = get_evaluate_energy(
            instance,
            precomputed_energies,
            p,
            objective="expectation",
            simulator=simulator,
        )
        initial_ar = eval_point(initial_point, eval_func, (minval, maxval), sense)
        print(f"{p=} {n=} {seed=} {initial_ar=}", flush=True)

        def objective_wrapper(params: Sequence[float], gradient: NDArray[np.float_]) -> float:
            if gradient.size > 0:
                raise NotImplementedError()
            return eval_func(params) * sense

        optimizer = nlopt.opt(method, len(initial_point))
        optimizer.set_ftol_rel(1e-13)
        optimizer.set_maxeval(int(maxfev))
        optimizer.set_initial_step(rhobeg)
        optimizer.set_min_objective(objective_wrapper)
        optimal_params = optimizer.optimize(np.array(initial_point))
        optimal_value = optimizer.last_optimum_value() * sense
        num_fun_evals = optimizer.get_numevals()
        exact_optimal_value = eval_func(optimal_params)
        ar = eval_point(optimal_params, eval_func, (minval, maxval), sense)
        print(f"    {optimal_value=} {exact_optimal_value=} {ar=}")


p=5 n=12 seed=0 initial_ar=0.6673376823128023
    optimal_value=9.836361345717636 exact_optimal_value=9.836361345717636 ar=0.6826008119702193
p=5 n=12 seed=1 initial_ar=0.7480974832087219
    optimal_value=4.783605218990238 exact_optimal_value=4.783605218990238 ar=0.7629046566237015
p=5 n=12 seed=2 initial_ar=0.763407497979244
    optimal_value=8.638649030606658 exact_optimal_value=8.638649030606658 ar=0.7767236566401157
p=5 n=12 seed=3 initial_ar=0.8101468585940048
    optimal_value=3.9541034091279883 exact_optimal_value=3.9541034091279883 ar=0.8153836388985273
p=5 n=12 seed=4 initial_ar=0.8190151512316447
    optimal_value=6.415603080070948 exact_optimal_value=6.415603080070948 ar=0.8258459307862607
p=5 n=12 seed=5 initial_ar=0.7709443695603642
    optimal_value=6.999744897812005 exact_optimal_value=6.999744897812005 ar=0.7771986811367316
p=5 n=12 seed=6 initial_ar=0.7255243095797446
    optimal_value=5.740526167543512 exact_optimal_value=5.740526167543512 ar=0.7461470773228223
p=5 n

KeyboardInterrupt: 